In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env file from the same directory as this notebook
# In Jupyter, the current working directory is typically where the notebook is located
try:
    env_file = Path('.env')
    if env_file.exists():
        load_dotenv(dotenv_path=env_file, override=True)
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in .env file")
        print(f"✅ Gemini API key loaded from .env file (first 10 chars: {google_api_key[:10]}...)")
        print(f"✅ Verified: GOOGLE_API_KEY is set in environment (length: {len(google_api_key)} chars)")
        print("ℹ️  ADK will use GOOGLE_API_KEY")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")

✅ Gemini API key loaded from .env file (first 10 chars: AIzaSyCxBj...)
✅ Verified: GOOGLE_API_KEY is set in environment (length: 39 chars)
ℹ️  ADK will use GOOGLE_API_KEY


In [2]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [4]:
server_params = StdioServerParameters(
    command="python",
    args=["/Users/ayoushchourasia/Documents/learning/5-Day-AI-Agents-Intensive-Course-with-Google/Day 2/examples/travel_mcp_server/server.py"],
    tool_filter=["get_weather_forecast", "search_flights"],  # Fixed: use underscore names
)

# Create MCP toolset
travel_mcp_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=server_params,
        timeout=30,
    )
)

In [5]:

PRICE_THRESHOLD = 10
def approve_trip_booking(cost: float, summary: str, tool_context: ToolContext) -> dict:
    if cost < PRICE_THRESHOLD:
        return {
            "status": "approved",
            "booking_id": f"BOOK-{int(cost)}-AUTO",
            "cost": cost,
            "message": f"✅ Trip auto-approved (under ${PRICE_THRESHOLD} threshold)",
            "details": summary
        }
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ This trip costs ${cost:.2f}. Do you want to approve?",
            payload={"price": cost, "summary": summary}
        )
        return {
            "status": "pending",
            "message": f"Trip requires approval (price: ${cost:.2f})"
        }
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "booking_id": f"BOOK-{int(cost)}-AUTO",
            "cost": cost,
            "message": f"✅ Trip approved (price: ${cost:.2f})",
            "details": summary
        }
    else:
        return {
            "status": "rejected",
            "message": "❌ Trip rejected by user"
        }

In [6]:
travel_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="travel_agent",
    instruction="""You are a travel planning assistant with approval workflow.
    
    IMPORTANT: Always use DEL (Delhi, India) as the departure airport unless user specifies otherwise.
    
    When user requests a trip:
    1. Extract destination city and travel dates from user query
    2. Call get_weather_forecast with destination and dates (format: "YYYY-MM-DD to YYYY-MM-DD")
    3. Call search_flights with:
       - origin: "DEL" (Delhi - DEFAULT, unless user specifies different departure city)
       - destination: appropriate airport code from this list:
         * Delhi=DEL, Mumbai=BOM, Bangalore=BLR, Indore=IDR
         * London=LHR, Paris=CDG, New York=JFK, Los Angeles=LAX
       - departure_date: in "YYYY-MM-DD" format
    4. Extract the first flight from the results and get its price
    5. Create trip summary text with destination, dates, weather, flight details, total price
    6. Call approve_trip_booking with the flight price as cost and the summary text
    7. Based on the response:
       - If "approved": Confirm booking with excitement
       - If "rejected": Acknowledge rejection professionally  
       - If "pending": Wait for human approval decision
    
    NEVER ask user for departure city - always use DEL as default!
    Always be friendly and enthusiastic about travel!
    """,
    tools=[travel_mcp_server, FunctionTool(func=approve_trip_booking)],
)

In [7]:
travel_app = App(
    name="travel_coordinator",
    root_agent=travel_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)
session_service = InMemorySessionService()
travel_runner = Runner(
    app=travel_app,
    session_service=session_service,
)

def check_for_approval(events):
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (part.function_call and 
                    part.function_call.name == "adk_request_confirmation"):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

def print_agent_response(events):
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

def extract_trip_details(events):
    """Extract weather, flight, and price info from events"""
    weather_info = None
    flight_info = None
    price = None
    trip_summary = None
    
    print("\n🔍 DEBUG: Extracting trip details from events...")
    
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                # Extract function responses
                if hasattr(part, 'function_response') and part.function_response:
                    fname = part.function_response.name
                    response = part.function_response.response
                    
                    print(f"   Found function response: {fname}")
                    
                    if fname == 'get_weather_forecast':
                        weather_info = response
                        print(f"   Weather data: {type(response)}")
                    elif fname == 'search_flights':
                        flight_info = response
                        print(f"   Flight data: {type(response)}, keys: {response.keys() if isinstance(response, dict) else 'N/A'}")
                
                # Extract function calls to see parameters
                if hasattr(part, 'function_call') and part.function_call:
                    fname = part.function_call.name
                    print(f"   Found function call: {fname}")
                    
                    if fname == 'approve_trip_booking':
                        # Get price and summary from the approval tool call
                        price = part.function_call.args.get('cost')
                        trip_summary = part.function_call.args.get('summary')
                        print(f"   Approval tool called with cost: {price}")
    
    print(f"   Final - Weather: {weather_info is not None}, Flight: {flight_info is not None}, Price: {price}\n")
    
    return weather_info, flight_info, price, trip_summary

def display_trip_summary(weather_info, flight_info, price, trip_summary):
    """Display formatted trip summary"""
    print("\n" + "="*70)
    print("📋 TRIP SUMMARY")
    print("="*70)
    
    if weather_info:
        print("\n🌤️  WEATHER:")
        if isinstance(weather_info, dict):
            temp = weather_info.get('temperature_range') or weather_info.get('temperature')
            conditions = weather_info.get('conditions') or weather_info.get('weather_conditions')
            packing = weather_info.get('packing_suggestions') or weather_info.get('packing')
            
            if temp:
                print(f"   Temperature: {temp}")
            if conditions:
                print(f"   Conditions: {conditions}")
            if packing:
                print(f"   Packing: {packing}")
        else:
            print(f"   {weather_info}")
    
    if flight_info:
        print("\n✈️  FLIGHT:")
        if isinstance(flight_info, dict):
            flights = flight_info.get('flights', [])
            if flights and len(flights) > 0:
                flight = flights[0]  # Show first flight
                print(f"   Airline: {flight.get('airline', 'N/A')}")
                print(f"   Route: {flight.get('origin', 'N/A')} → {flight.get('destination', 'N/A')}")
                print(f"   Departure: {flight.get('departure_time', 'N/A')}")
                print(f"   Arrival: {flight.get('arrival_time', 'N/A')}")
                print(f"   Duration: {flight.get('duration', 'N/A')}")
                print(f"   Price: ${flight.get('price', 'N/A')}")
        else:
            print(f"   {flight_info}")
    
    if price:
        print(f"\n💰 TOTAL PRICE: ${price}")
    
    if trip_summary:
        print(f"\n📝 SUMMARY:\n   {trip_summary}")
    
    print("="*70)

def create_approval_response(approval_info, approved):
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", 
        parts=[types.Part(function_response=confirmation_response)]
    )

/var/folders/m5/vw686v2x6xj9k9z7211qm4jh0000gn/T/ipykernel_55378/3716989590.py:4: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [8]:
async def run_travel_agent_interactive():
    """
    Interactive travel agent with iterative search.
    
    User can approve (Y) or reject and try new destination (N [City] [Dates]).
    """
    
    print("\n" + "="*70)
    print("🌍 Welcome to Travel Agent with LRO Practice!")
    print("="*70)
    print("\nFormat: 'Plan trip to [City] [StartDate] to [EndDate]'")
    print("Example: 'Plan trip to Mumbai 1st Jan 2026 to 5th Jan 2026'")
    print("\nAfter seeing results:")
    print("  - Type 'Y' to approve and book")
    print("  - Type 'N [City] [StartDate] to [EndDate]' to try different destination")
    print("="*70 + "\n")
    
    query = input("You > ")
    
    while True:
        print(f"\n{'─'*70}")
        
        # Create new session for this search
        session_id = f"trip_{uuid.uuid4().hex[:8]}"
        await session_service.create_session(
            app_name="travel_coordinator",
            user_id="traveler",
            session_id=session_id
        )
        
        query_content = types.Content(role="user", parts=[types.Part(text=query)])
        events = []
        
        # Send query to agent
        async for event in travel_runner.run_async(
            user_id="traveler",
            session_id=session_id,
            new_message=query_content
        ):
            events.append(event)
        
        # Check if agent paused
        approval_info = check_for_approval(events)
        
        if approval_info:
            # Agent paused - expensive trip
            print(f"\n⏸️  Agent paused for approval:")
            
            # Extract and display trip details
            weather_info, flight_info, price, trip_summary = extract_trip_details(events)
            display_trip_summary(weather_info, flight_info, price, trip_summary)
            
            # Also show any agent text
            print_agent_response(events)
            
            # Get real user input
            user_input = input("\nYou > ").strip()
            
            if user_input.upper() == "Y":
                # Approve and book
                print("\n✅ Approving trip...\n")
                
                async for event in travel_runner.run_async(
                    user_id="traveler",
                    session_id=session_id,
                    new_message=create_approval_response(approval_info, True),
                    invocation_id=approval_info["invocation_id"],  # CRITICAL
                ):
                    if event.content and event.content.parts:
                        for part in event.content.parts:
                            if part.text:
                                print(f"Agent > {part.text}")
                
                print(f"\n{'─'*70}")
                print("🎉 Booking complete! Thanks for using Travel Agent.")
                print("="*70 + "\n")
                break
                
            elif user_input.upper().startswith("N "):
                # Reject and search new destination
                print("\n❌ Rejecting current trip...\n")
                
                # Complete rejection
                async for event in travel_runner.run_async(
                    user_id="traveler",
                    session_id=session_id,
                    new_message=create_approval_response(approval_info, False),
                    invocation_id=approval_info["invocation_id"],
                ):
                    pass  # Just finalize rejection
                
                # Parse new query
                new_location_dates = user_input[2:].strip()  # Remove "N "
                query = f"Plan trip to {new_location_dates}"
                print(f"🔄 Searching new destination: {new_location_dates}\n")
                continue  # Loop back
                
            else:
                print("\n❌ Invalid input. Use 'Y' or 'N [City] [Dates]'")
                break
        else:
            # Auto-approved (cheap trip)
            print("\n✅ Trip auto-approved (under price threshold)!")
            
            # Extract and display trip details for auto-approved trips too
            weather_info, flight_info, price, trip_summary = extract_trip_details(events)
            display_trip_summary(weather_info, flight_info, price, trip_summary)
            
            # Show agent response
            print_agent_response(events)
            print(f"\n{'─'*70}")
            print("🎉 Booking complete! Thanks for using Travel Agent.")
            print("="*70 + "\n")
            break

print("✅ Interactive workflow ready")

✅ Interactive workflow ready


In [9]:
await run_travel_agent_interactive()


🌍 Welcome to Travel Agent with LRO Practice!

Format: 'Plan trip to [City] [StartDate] to [EndDate]'
Example: 'Plan trip to Mumbai 1st Jan 2026 to 5th Jan 2026'

After seeing results:
  - Type 'Y' to approve and book
  - Type 'N [City] [StartDate] to [EndDate]' to try different destination


──────────────────────────────────────────────────────────────────────


/Users/ayoushchourasia/Documents/learning/5-Day-AI-Agents-Intensive-Course-with-Google/env/lib/python3.13/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



⏸️  Agent paused for approval:

🔍 DEBUG: Extracting trip details from events...
   Found function call: get_weather_forecast
   Found function call: search_flights
   Found function response: get_weather_forecast
   Weather data: <class 'dict'>
   Found function response: search_flights
   Flight data: <class 'dict'>, keys: dict_keys(['content', 'isError'])
   Found function call: approve_trip_booking
   Approval tool called with cost: 744
   Found function call: adk_request_confirmation
   Found function response: approve_trip_booking
   Final - Weather: True, Flight: True, Price: 744


📋 TRIP SUMMARY

🌤️  WEATHER:

✈️  FLIGHT:

💰 TOTAL PRICE: $744

📝 SUMMARY:
   Trip to London from January 1st, 2026 to January 10th, 2026. Weather: Temperatures ranging from 1.6°C to 14.4°C with sunny, cloudy, and dreary conditions. Packing suggestions include layers, a medium to warm jacket, comfortable walking shoes, sunscreen, and sunglasses. Flight: Lufthansa (LH586) from Delhi (DEL) to London (LH

/Users/ayoushchourasia/Documents/learning/5-Day-AI-Agents-Intensive-Course-with-Google/env/lib/python3.13/site-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(



❌ Rejecting current trip...



/Users/ayoushchourasia/Documents/learning/5-Day-AI-Agents-Intensive-Course-with-Google/env/lib/python3.13/site-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


🔄 Searching new destination: mumbai 1st December 2025 to 5th December 2025


──────────────────────────────────────────────────────────────────────



⏸️  Agent paused for approval:

🔍 DEBUG: Extracting trip details from events...
   Found function call: get_weather_forecast
   Found function call: search_flights
   Found function response: get_weather_forecast
   Weather data: <class 'dict'>
   Found function response: search_flights
   Flight data: <class 'dict'>, keys: dict_keys(['content', 'isError'])
   Found function call: approve_trip_booking
   Approval tool called with cost: 388
   Found function call: adk_request_confirmation
   Found function response: approve_trip_booking
   Final - Weather: True, Flight: True, Price: 388


📋 TRIP SUMMARY

🌤️  WEATHER:

✈️  FLIGHT:

💰 TOTAL PRICE: $388

📝 SUMMARY:
   Destination: Mumbai
Dates: December 1st, 2025 to December 5th, 2025
Weather: Sunny, with temperatures between 17.3°C and 35.4°C.
Flight: Lufthansa, $388, departing December 1st, 2025 at 09:15, arriving December 1st, 2025 at 11:15.
Agent > The weather in Mumbai from December 1st to December 5th, 2025 will be sunny with temper